## Preambule

In [1]:
import numpy as np
from tqdm import tqdm
from importlib import reload
import xarray as xr
import pandas as pd
import scipy
from scipy.optimize import curve_fit
from scipy.signal import savgol_filter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Sobol analysis
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Read stuff

In [2]:
path_data = "K:/data/DataUpdate_ongoing/"

In [14]:
xr_dataread = xr.open_dataset(path_data + "xr_dataread.nc")
xr_sobol = xr.open_dataset(path_data + "xr_sobol.nc").sel(Time = np.arange(2030, 2101, 5))
all_countries_iso = np.load(path_data + "all_countries.npy", allow_pickle=True)
all_regions_iso = np.load(path_data + "all_regions.npy", allow_pickle=True)

## Process stuff

In [15]:
dim_unc_globe = ['Temperature', 'Timing', 'NegEmis']
dim_unc_fairness = ['variable', 'Convergence_year', 'Historical_startyear', 'Discount_factor'] 
dim_unc_choices = ['Scenario', 'NonCO2red', 'Risk']

vals = np.zeros(shape=(3, len(xr_sobol.Time), len(all_countries_iso)))
for dim_i, dim in enumerate([dim_unc_globe, dim_unc_fairness, dim_unc_choices]):
    vals[dim_i] = np.array(xr_sobol.sel(Factor = dim, Region=all_countries_iso).sum(dim='Factor').to_array())[0]

In [16]:
for factor in np.array(xr_sobol.Factor):
    print(factor, float(xr_sobol.sel(Time=2040, Region='USA', Factor=factor).Sobol_index))

factors = ['variable', 'Discount_factor', 'Historical_startyear', 'Convergence_year',
           'NonCO2red', 'Risk', 'Scenario', 
           'Temperature', 'Timing', 'NegEmis']
factor_names = ['Allocation rule', 'Discount factor', 'Historical startyear', 'Convergence year', 
                 'Non-CO<sub>2</sub> reduction', 'Climate sensitivity','Socio-economic scenario',
                'Peak temperature', 'Timing of action', 'Negative emissions']
factor_colors = ['goldenrod', 'salmon', 'tomato', 'darkred', 
                  'dodgerblue', 'steelblue', 'navy', 
                 'lightgreen', 'yellowgreen', 'forestgreen']

variable 0.2560505474773743
Convergence_year 0.008800339024167785
NegEmis 0.0015934101789112768
NonCO2red 0.06153788769868976
Temperature 0.4426355151205055
Risk 0.07483548033561621
Timing 0.0005847388454770884
Scenario 0.009812337903578779
Discount_factor 0.09856922144369448
Historical_startyear 0.0455805219719849


In [17]:
vals[:, 0, np.where(all_countries_iso == 'USA')[0][0]]

array([0.68819912, 0.15667392, 0.15512696])

In [18]:
mins = np.nanmin(vals, axis=2)
maxs = np.nanmin(vals, axis=2)
mins = (np.array(mins)*100).round(0).astype(int).astype(str)
maxs = (np.array(maxs)*100).round(0).astype(int).astype(str)

In [56]:
df_regions = pd.read_csv("X:/user/dekkerm/Data/AllCountries.csv")
continents = []
bubblecolors = []
texts = []
for cty_i, cty in enumerate(all_countries_iso):
    if cty == 'USA':
        bubblecolors.append('plum')
        texts.append(cty)
    elif cty == 'IND':
        bubblecolors.append('forestgreen')
        texts.append(cty)
    elif cty == 'BRA':
        bubblecolors.append('goldenrod')
        texts.append(cty)
    elif cty == 'CHN':
        bubblecolors.append('tomato')
        texts.append(cty)
    elif cty == 'ZAF':
        bubblecolors.append('steelblue')
        texts.append(cty)
    elif cty == 'UGA':
        bubblecolors.append('lightgreen')
        texts.append(cty)
    elif cty == 'TUR':
        bubblecolors.append('brown')
        texts.append(cty)
    elif cty == 'RWA':
        bubblecolors.append('dodgerblue')
        texts.append(cty)
    else:
        bubblecolors.append('silver')
        texts.append('')
continents = np.array(continents)
bubblecolors = np.array(bubblecolors)
texts = np.array(texts)

pops = np.array(xr_dataread.Population.sel(Region=all_countries_iso, Time=2020, Scenario='SSP2'))
pops[np.isnan(pops)] = 1
pops_n = pops/np.nanmax(pops)

## Plot

In [20]:
val_i = xr_sobol.sel(Region='USA', Factor='variable').Sobol_index

In [21]:
def draw_line(fig, name, x1, x2, y1, y2, color):
    fig.add_shape(type="line", xref='paper', yref='paper',
                  x0=x1,
                  y0=y1,
                  x1=x2,
                  y1=y2,
                  line=dict(color=color, width=3))

In [74]:
fig = make_subplots(rows=4, cols=3, specs=[[{"type": "scatterternary", "colspan": 2, "rowspan": 4}, {}, {}],
                                           [{}, {}, {}],
                                           [{}, {}, {}],
                                           [{}, {}, {}]],
                    horizontal_spacing = 0, vertical_spacing=0.05)

# Panel a
for i in np.arange(0., 1.01, 0.25):
    args = dict(mode='lines', showlegend=False, hoverinfo='skip',
                                                    marker={'size': 15,
                                                            'color': 'black',
                                                            'opacity': 1,#0.9-i*0.03,
                                                            'line' :dict(width=0., color='silver')},
                                                    line={'width': 0.33, #np.mean(s[i:i+2])*0.5,
                                                            'color': 'black',
                                                            'dash': 'dot'},
                                                    textfont=dict(size=5,
                                                            color='black'))
    fig.add_trace(go.Scatterternary(a=[i, i], b=[0, 1-i], c=[1-i, 0], **args), 1, 1)
    fig.add_trace(go.Scatterternary(b=[i, i], a=[0, 1-i], c=[1-i, 0], **args), 1, 1)
    fig.add_trace(go.Scatterternary(c=[i, i], b=[0, 1-i], a=[1-i, 0], **args), 1, 1)

for cty_i, cty in enumerate(all_countries_iso):
        fig.add_trace(go.Scatterternary(a=vals[0, :, cty_i],
                                        b=vals[1, :, cty_i],
                                        c=vals[2, :, cty_i],
                                        name=cty,
                                                        mode='lines+markers+text',
                                                        showlegend=False,
                                                        text=[texts[cty_i]]+['']*(len(vals[0])-1),
                                                        marker={'size': 6,
                                                                'color': bubblecolors[cty_i],
                                                                'opacity': 0.9,#0.9-i*0.03,
                                                                'line' :dict(width=0, color='black')},
                                                        line={'width': 0.5, #np.mean(s[i:i+2])*0.5,
                                                                'color': 'silver'},
                                                        textfont=dict(size=(pops_n[cty_i])**0.4*25+1,
                                                                color='black')), 1, 1)
for cty in ['USA', 'CHN', 'BRA', 'ZAF', 'IND', 'UGA', 'RWA', 'TUR']:
        cty_i = np.where(all_countries_iso == cty)[0][0]
        fig.add_trace(go.Scatterternary(a=vals[0, :, cty_i],
                                        b=vals[1, :, cty_i],
                                        c=vals[2, :, cty_i],
                                        name=cty,
                                                        mode='lines+markers+text',
                                                        showlegend=False,
                                                        text=[texts[cty_i]]+['']*(len(vals[0])-1),#["'., str(i-2000) for i in [2040, 2050, 2060, 2070, 2080, 2090]],
                                                        marker={'size': [(pops_n[cty_i])**0.4*60+4]+[15]*(len(vals[0])-1),
                                                                'color': bubblecolors[cty_i],
                                                                'opacity': 0.9,#0.9-i*0.03,
                                                                'line' :dict(width=0.5, color='white')},
                                                        line={'width': 3, #np.mean(s[i:i+2])*0.5,
                                                                'color':bubblecolors[cty_i]},
                                                        textfont=dict(size=(pops_n[cty_i])**0.4*25+4,
                                                                color='black')), 1, 1)

fig.update_layout({
'plot_bgcolor':'rgb(243, 243, 243)',
'ternary':
{'sum':1,
'bgcolor':'white',
    # "paper_bgcolor":'rgba(0,0,0,0)',
    # "plot_bgcolor":'rgba(0,0,0,0)',
'aaxis':{'title': '', 'min': 0., 'titlefont':{'size': 25},#Climate target<br>
        'linewidth':0, 'ticks':'outside',
        'tickmode':'array','tickvals':np.arange(0, 1.01, 0.25), 'ticktext':[str(int(i*100))+'%' for i in np.arange(0, 1.01, 0.25)], 'tickfont':{'size':15}},
'baxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},# &nbsp; &nbsp; &nbsp;#Model &nbsp; &nbsp;
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':np.arange(0, 1.01, 0.25), 'ticktext':[str(int(i*100))+'%' for i in np.arange(0, 1.01, 0.25)],'tickangle':60, 'tickfont':{'size':15}},
'caxis':{'title': '', 'min': 0, 'titlefont':{'size': 25},#Other<br>scenario assumptions
        'linewidth':2, 'ticks':'outside',
        'tickmode':'array','tickvals':np.arange(0, 1.01, 0.25), 'ticktext':[str(int(i*100))+'%' for i in np.arange(0, 1.01, 0.25)],'tickangle':-60, 'tickfont':{'size':15}}}})

# Panel b-d
for cty_i, cty in enumerate(['USA', 'CHN', 'TUR', 'RWA']):
        for factor_i, factor in enumerate(factors):
                col = factor_colors[factor_i]
                val_i = xr_sobol.sel(Region=cty, Factor=factor).Sobol_index
                if factor_i == 0: fill='tozeroy'
                else: fill = 'tonexty'
                fig.add_trace(go.Scatter(x=xr_sobol.Time,
                                        y=val_i,
                                        mode='lines',
                                        marker={'color':col},
                                        fill=fill,
                                        name=factor_names[factor_i],
                                        showlegend=[True, False, False, False, False, False, False][cty_i],
                                        stackgroup='one'
                                        ), range(1, 6)[cty_i], 3)

fig.update_geos(visible=False,
                showlakes=True,
                lakecolor='rgb(255, 255, 255)',
                projection_type='natural earth',
                resolution=110,
                showcoastlines=False,)

fig.update_layout(
    legend=dict(
        yanchor="top",
        y=0.29,
        xanchor="left",
        x=0.08,
        font=dict(size=15, color="black"),
    ),
    #template='ggplot2',
    height=900,
    width=1600,
)
fig.update_xaxes(visible=False, row=1, col=3)
fig.update_xaxes(visible=False, row=2, col=3)
fig.update_xaxes(visible=False, row=3, col=3)

args = dict(xref="paper", yref="paper", showarrow=False)
annotations = [
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.98, x=0.0, text="<b>(a) Relative impact of<br>the allocation dimensions</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=1.04, x=0.85, text="<b>(b) United States</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.78, x=0.77, text="<b>(c) China</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.49, x=0.784, text="<b>(d) Turkey</b>"),
    dict(xref='paper', yref='paper', ax=0, align='left', ay=-0, showarrow=False, font=dict(color='black', size=24), y=0.21, x=0.79, text="<b>(e) Rwanda</b>"),
    dict(text="Global targets", x=0.28, y=1.12, **args, font=dict(color='black', size=25)),
    dict(text="temperature, negative emissions, timing", x=0.23, y=1.08, **args, font=dict(color='darkgray', size=16)),
    dict(text="Equity", x=0, y=0.13, **args, font=dict(color='black', size=25)),
    dict(text="allocation rule, discounting,<br>historical startyear<br>convergence year", x=-0.05, y=0.04, **args, font=dict(color='darkgray', size=16)),
    dict(text="Physical and social<br>uncertainties", x=0.55, y=0.08, **args, font=dict(color='black', size=25)),
    dict(text="scenario, non-CO2,<br>climate sensitivity", x=0.55, y=0.02, **args, font=dict(color='darkgray', size=16))
]
for ann in annotations:
    fig.add_annotation(**ann)

#draw_line(fig, "", 0.65, 0.24, 1, 0.4, "black")

#fig.update_layout(margin={"r":20,"t":50,"l":100,"b":50})
fig.update_layout(
       legend=dict(x=0.5, y=0.95),
    paper_bgcolor='white',  # Set the background color
    plot_bgcolor='white'
)
fig.write_image("K:/Code/EffortSharing/Figures/Paper_FairShares/Figure_3.png", scale=3)

In [70]:
np.array(xr_sobol.sel(Time=2060, Region=['USA', 'CHN', 'IND'], Factor='Temperature').Sobol_index)

array([0.05014223, 0.34787901, 0.30074261])